In [2]:
from urllib import *
from html import unescape
import requests
import json 
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
import requests
import time
import bs4
from bs4 import BeautifulSoup
import re
import time


def download(method, url, 
             param = None, data = None, 
            timeout = 1,maxretries = 3):
    
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    #headers = None
    
    try:
        resp = requests.request(method, url, params= param, data=data, headers=headers)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0: 
            time.sleep(timeout) # param에따라 몇초 기다릴지 결정 할 수가 있다.
            print(maxretries) # 재귀적으로 자기 자신을 부르게 코드를 짜면 된다.
            resp = download(method,
                            url, param = param, data=data,
                            timeout=timeout,
                            maxretries = maxretries - 1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")
    ## 정규식 re.search(".+?\.tistory.com", "http:\\adam24eve.tistory.com/") #짧게매칭되는거 ? 레이지 

def parseURL(seed):
    html = download("get",seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    ## naver iframe html안에 html 또들어있어서
   # if len(dom.select("#mainFrame")) < 1:  ## 메인이랑, 첫페이지만 없음
   #     return []
    
   
   # seed = requests.compat.urljoin(seed,dom.select("#mainFrame")[0]["src"])
  #  print("-=================================================")
   # print(seed)
  #  print("-=================================================")
   # html = download("get",seed)
   # dom = BeautifulSoup(html.text, "lxml")
    

    return  [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") if  _.has_attr("href") and len(_["href"]) >3 ]

##and checkBlog(requests.compat.urljoin(seed, _["href"]))






In [3]:
## url을 pk값으로 삼으면 안됨 문자열로 삼으면 안된다  => 해시값을 못만든다.  인덱싱하는데 느려진다. pk제대로 못걸르고
# 언제 엑세싱했는지 처음방문했는지 data값 필요
# seen 내가 본앤지 아닌지 검사하는 
# =>> 최소 4개 컬럼 id, url, seen, data
# 특정 url 해당되는 애들만 쓰고싶으면 

## 스크랩파이 내부구조를 배우는중

In [4]:
table1 = id, url, seen, date
# url path만 있고 파라메터값 쿼리 넣어서 돌린다. t1과 , t2를 조인
table2 = id, netloc, date
# 특정도메인만 관리하는 테이블

url = netloc + path + param

NameError: name 'url' is not defined

In [5]:
import sqlite3
con = sqlite3.connect("bot6.db")
cur = con.cursor()

In [7]:
cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT ,
        seen BOOLEAN DEFAULT FALSE NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );

""")
con.commit()

In [8]:
## 실제 입력하는 PATHH PARAM 본적있으면 seen => true

In [11]:
# Seed URLs  -> DB Insert 
url = "https://www.google.com/search"
html = download("get",url, param={"q":"박보영"})
dom = BeautifulSoup(html.text, "lxml")

for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
    
    _urlparse = requests.compat.urlparse(href)
  
    netloc = "://".join(_urlparse[:2])  ## 0과 1 합침
    ## print(netloc)
    cur.execute("SELECT id FROM table2 WHERE netloc = ? LIMIT 0,1", [netloc]) ## re!! limit 0,1 이랑 fetchone()랑 차이가 뭐냐??
    netlocID = cur.fetchone()
    print(netlocID)
    

    


    ############################################################################
    
    if not netlocID: 
        cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
        con.commit()
        cur.execute("SELECT id FROM table2 WHERE netloc=?  LIMIT 0,1", [netloc])

        netlocID =cur.fetchone()
        print(netlocID[0])
        print("--")
    cur.execute("INSERT INTO table1(table2_id, path, param) VALUES(?,?,?)", [netlocID[0], _urlparse[2], _urlparse[4]])
    con.commit()  ## 여기까지하면 큐가 생성되어있다.
    print(cur.lastrowid, netlocID)
       # DB Select Limit 0,1 하면 SELECT 해서 여러개 있는거중에서  위에 있는거 하나만 가져오는거

https://ko.wikipedia.org
(1,)
9 (1,)
https://ko.wikipedia.org
(1,)
10 (1,)
https://news.joins.com
(2,)
11 (2,)
https://twitter.com
(3,)
12 (3,)
https://movie.daum.net
(4,)
13 (4,)
http://m.cafe.daum.net
(5,)
14 (5,)
https://www.msn.com
(6,)
15 (6,)
https://www.msn.com
(6,)
16 (6,)


In [ ]:
## 여기부터 re!! db만들고, seed를 튜플로 둬서 depth , inbound 까지도 db에 저장하는것까지

In [10]:
i = 0
while True:
    cur.execute("""SELECT table1.id, table2.netloc, table1.path, table1.param
                   FROM table1
                   JOIN table2
                  ON table1.table2_id=table2.id
                   WHERE table1.seen='FALSE'
                   
                   ORDER BY table1.date ASC 
                   ; """)
    time.sleep(1)
    seed = cur.fetchone()
    if not seed or i > 100:
        break;
    i+1
    print(seed)

(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB%B3%B4%EC%98%81', '')
(1, 'https://ko.wikipedia.org', '/wiki/%EB%B0%95%EB

KeyboardInterrupt: 

In [65]:
def parseURL(seed):
    html = download("get",seed[0])
    dom = BeautifulSoup(html.text, "lxml")
    

    return  [(requests.compat.urljoin(seed[0], _["href"]),seed[1]+1)
             for _ in dom.find_all("a") 
             if seed[1] < 2 and  _.has_attr("href") and len(_["href"]) >3 and checkBlog(requests.compat.urljoin(seed[0], _["href"]))]
# 뎁스를 2까지만 하겠다.




In [ ]:
queue = list() ##하나 생김
queue.extend( [(_["href"],1) for _ in dom.select(".LC201b") if checkBlog(_["href"])])
seen = list()
while queue :
    baseURL = queue.pop(0)  ##지금 너비우선
                            ##pop(0)을 깊이우선 -1로 바꿔서 depth 를 정해놓고 or 특정도메인내에서만 => 포커스드크롤링
    print(baseURL)
    seen.append(baseURL)
    # 슬립 
    time.sleep(1)
    linkList = parseURL(baseURL) # 중복된 url 계속 생김 re??
 
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))


In [ ]:
cur.close()
con.close()